In [ ]:
import pickle
import re
import pandas as pd

### Helper function

In [ ]:
companies = [
    'arko corp', 'j jill inc', 'abercrombie', 'academy sports and outdoors inc',
    'advance auto parts inc', 'koninklijke ahold delhaize', 'albertsons companies inc',
    'alimentation couche tard inc', 'amazon com inc', 'asbury automotive group inc',
    'autonation inc', 'autozone inc', 'bjs wholesale club holdings inc', 'bath body works inc',
    'best buy co inc', 'burlington stores inc', 'camping world holdings inc',
    'canada goose holdings inc', 'carvana co', 'the container store group inc',
    'costco wholesale corporation', 'dicks sporting goods inc', 'dollar general corporation',
    'dollar tree inc', 'wisdomTree international largeCap dividend fund', 'floor decor holdings inc',
    'foot locker inc', 'franchise group inc', 'the gap inc', 'genuine parts company',
    'group 1 automotive inc', 'the home depot inc', 'ingles markets incorporated',
    'kohls corporation', 'the kroger co', 'leslies inc', 'qurate retail inc',
    'lithia motors inc', 'lowes companies inc', 'macys inc', 'capri holdings limited',
    'mister car wash inc', 'murphy usa inc', 'national vision holdings inc',
    'nordstrom inc', 'oreilly automotive inc', 'penske automotive group inc',
    'petco health and wellness company inc', 'ralph lauren corporation', 'rh',
    'ross stores inc', 'albertsons companies inc', 'sonic automotive inc',
    'the tjx companies inc', 'tapestry inc', 'target corporation', 'torrid holdings inc',
    'tractor supply company', 'valvoline inc', 'victorias secret co',
    'walgreens boots alliance inc', 'walmart inc'
]

company_to_ticker = {
    'abercrombie': 'ANF',
    'academy sports and outdoors inc': 'ASO',
    'advance auto parts inc': 'AAP',
    'albertsons companies inc': 'ACI',
    'alimentation couche tard inc': 'ATD',
    'amazon com inc': 'AMZN',
    'arko corp': 'ARKO',
    'asbury automotive group inc': 'ABG',
    'autonation inc': 'AN',
    'autozone inc': 'AZO',
    'bath body works inc': 'BBWI',
    'best buy co inc': 'BBY',
    'bjs wholesale club holdings inc': 'BJ',
    'burlington stores inc': 'BURL',
    'camping world holdings inc': 'CWH',
    'canada goose holdings inc': 'GOOS',
    'capri holdings limited': 'CPRI',
    'carvana co': 'CVNA',
    'costco wholesale corporation': 'COST',
    'dicks sporting goods inc': 'DKS',
    'dollar tree inc': 'DLTR',
    'floor decor holdings inc': 'FND',
    'foot locker inc': 'FL',
    'franchise group inc': 'FRG',
    'genuine parts company': 'GPC',
    'ingles markets incorporated': 'IMKTA',
    'j jill inc': 'JILL',
    'kohls corporation': 'KSS',
    'koninklijke ahold delhaize': 'AD',
    'leslies inc': 'LESL',
    'lithia motors inc': 'LAD',
    'lowes companies inc': 'LOW',
    'macys inc': 'M',
    'mister car wash inc': 'MCW',
    'murphy usa inc': 'MUSA',
    'national vision holdings inc': 'EYE',
    'nordstrom inc': 'JWN',
    'oreilly automotive inc': 'ORLY',
    'penske automotive group inc': 'PAG',
    'petco health and wellness company inc': 'WOOF',
    'qurate retail inc': 'QRTEA',
    'ralph lauren corporation': 'RL',
    'rh': 'RH',
    'ross stores inc': 'ROST',
    'sonic automotive inc': 'SAH',
    'tapestry inc': 'TPR',
    'target corporation': 'TGT',
    'the gap inc': 'GPS',
    'the kroger co': 'KR',
    'torrid holdings inc': 'CURV',
    'tractor supply company': 'TSCO',
    'valvoline inc': 'VVV',
    'victorias secret co': 'VSCO',
    'walgreens boots alliance inc': 'WBA',
    'walmart inc': 'WMT'
}

In [ ]:
def find_matching_organizations(org_text):
    matches = []
    for company in companies:
        pattern = rf'\b{re.escape(company)}\b'
        if re.search(pattern, org_text, re.IGNORECASE):
            matches.append(company)
    return ', '.join(matches) if matches else None

In [ ]:
def check_multiple_companies(row):
    companies = row.split(',')
    return len(companies) > 1

In [ ]:
def split_companies(row):
    companies = row['Companies'].split(';')
    new_rows = []
    for company in companies:
        new_row = row.copy()
        new_row['Companies'] = company
        new_rows.append(new_row)
    return new_rows

### Main

In [ ]:
#Load data
df = pd.read_csv('test.csv')

#get matching companies
df['Companies'] = df['Organizations'].apply(find_matching_organizations)

#check if multiple companies
df = df.assign(Companies=df['Companies'].str.split(', ')).explode('Companies').reset_index(drop=True)

#split V2Tone
df[['Tone1', 'Tone2', 'Tone3', 'Tone4', 'Tone5', 'Tone6', 'Tone7']] = df['V2Tone'].str.split(',', expand=True)

#convert to datetime
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d%H%M%S')

#add quarter
df['Quarter'] = df['DATE'].dt.to_period('Q')

#convert to numeric
df[['Tone1', 'Tone2', 'Tone3', 'Tone4', 'Tone5', 'Tone6', 'Tone7']] = df[['Tone1', 'Tone2', 'Tone3', 'Tone4', 'Tone5', 'Tone6', 'Tone7']].apply(pd.to_numeric)

#drop unnecessary columns
df.drop(columns=['V2Tone', 'Organizations'], inplace=True)

#group by companies and quarter
quarterly_avg = df.groupby(['Companies', 'Quarter'])[['Tone1', 'Tone2', 'Tone3', 'Tone4', 'Tone5', 'Tone6', 'Tone7']].mean().reset_index()



In [ ]:
# Create a DataFrame containing all quarters from 2009Q4 to 2024Q2
all_quarters = pd.date_range(start='2009Q4', end='2024Q2', freq='Q')
all_quarters = all_quarters.to_period('Q').astype(str)
all_quarters_df = pd.DataFrame(all_quarters, columns=['Quarter'])

# Get all unique company names from the quarterly_avg DataFrame
companies = quarterly_avg['Companies'].unique()

# Create a list to hold DataFrames for each company with all quarters
df_full_list = []
for company in companies:
    df_full = all_quarters_df.copy()
    df_full['Companies'] = company
    df_full_list.append(df_full)

# Concatenate all DataFrames in the list into one DataFrame
df_full = pd.concat(df_full_list, ignore_index=True)

# Ensure the 'Quarter' column type is consistent across DataFrames
quarterly_avg['Quarter'] = quarterly_avg['Quarter'].astype(str)
df_full['Quarter'] = df_full['Quarter'].astype(str)

# Merge the full quarters DataFrame with the quarterly_avg DataFrame
# using a left join to include all combinations of companies and quarters
df_full = pd.merge(df_full, quarterly_avg, on=['Companies', 'Quarter'], how='left')

# Rename the columns to more descriptive names
df_full.rename(columns={'Tone1': 'Tone', 'Tone2': 'Positive Score', 'Tone3': 'Negative Score', 'Tone4': 'Polarity Score', 'Tone5': 'Activity Reference Density', 'Tone6': 'Self/Group Reference Density', 'Tone7': 'Word Count '}, inplace=True)

# Replace company names with stock ticker symbols using the mapping dictionary
df_full['Companies'] = df_full['Companies'].replace(company_to_ticker)
